# ChatBOT for Imarticus Learning

Installing the necessary libraries

In [7]:
!pip install PyPDF2
!pip install openai
!pip install langchain
!pip install -U langchain-community
!pip install tiktoken
!pip install faiss-cpu
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.2 MB/s eta 0:00:00


In [9]:
import os
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [38]:
# Setting OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [30]:
pdfreader = PdfReader("ilovepdf_merged.pdf")

In [39]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [40]:
raw_text

'POSTGRADUATE\nPROGRAM IN\nBANKING AND\nFINANCE\nGuaranteed interview opportunities\nIdeal for launching into entry-level\npositions in the banking and ﬁnancial\nservices industry\n4-month weekday program with\n300+ contact hours\nwww.imarticus.orgEarn an NISM Certiﬁcation\nin mutual fundsIndustry Landscape\nwww.imarticus.org 02\nGrowth\nOpportunities\nIn The \nBanking \nAnd Financial\nServices\nIndustryCAREER TRAJECTORY\nAVERAGE SALARY0-1 YEARS10+ YEARS\n8 YEARS\n6 YEARS\n4 YEARS\nDEPUTY MANAGER\nBRANCH MANA GER\nREGIONAL MANA GER\nDIVISION HEADBANK\nOFFICER\nBANK/BRANCH\nOFFICERDEPUTY\nMANA GERBRANCH\nMANA GERDIVISION HEAD\n` 2-4\nLAKHS` 5-6\nLAKHS` 15-20\nLAKHS` 30+\nLAKHS\nNBFCs and private banks are expected to hire 70,000 in the next three years.\nHDFC Bank aims to reach 200,000 villages in the next 2 years and plans to hire\nover 2500 people. Similarly, institutions such as ICICI and Kotak also plan to\nrecruit in big numbers in the coming years.Program Overview\nCurriculum\nwww

In [42]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 950,
    chunk_overlap  = 250,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [43]:
len(texts)

181

In [44]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [45]:
document_search = FAISS.from_texts(texts, embeddings)

In [46]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [47]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [56]:
query = "topics covered in Stats in Data Science"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

'\nIntroduction to Statistics, Probability Theory, Probability Distributions, Hypothesis Testing, Statistical Tests'

In [57]:
def query_stats_topics(question):
    query = "topics covered in Stats in Data Science"
    docs = document_search.similarity_search(query)
    answer = chain.run(input_documents=docs, question=question)
    return answer

iface = gr.Interface(fn=query_stats_topics,
                     inputs="text",
                     outputs="text",
                     title="Welcome",
                     description="Hii, Im Imarti a chatbot dedicated to help you!.")

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://642c1d79b949efa964.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [60]:
!pip install --upgrade gradio

import gradio as gr

def query_stats_topics(question):
    query = "topics covered in Stats in Data Science"
    docs = document_search.similarity_search(query)
    answer = chain.run(input_documents=docs, question=question)
    return answer

# Function to update chat history
def respond(message, history):
    if history is None:
        history = []
    response = query_stats_topics(message)
    history.append((message, response))
    return history, history

# Create the Gradio interface
iface = gr.Interface(
    fn=respond,
    inputs=[gr.components.Textbox(lines=1, placeholder="Ask a question..."), gr.components.State()], # Use gr.components for input elements
    outputs=[gr.components.Chatbot(), gr.components.State()], # Use gr.components for output elements
    title="Welcome",
    description="Hi, I'm Imarti, a chatbot dedicated to helping you!"
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://eda5d7928afb04ff9d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
